<a href="https://colab.research.google.com/github/karaage0703/karaage-ai-book/blob/master/ch03/03_karaage_ai_book_generate_text_markov_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルコフ連鎖による文章生成

## 教師データのダウンロード

In [1]:
!wget https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
!unzip 2093_ruby_28087.zip

--2020-11-16 13:57:56--  https://github.com/aozorabunko/aozorabunko/raw/master/cards/000096/files/2093_ruby_28087.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip [following]
--2020-11-16 13:57:56--  https://raw.githubusercontent.com/aozorabunko/aozorabunko/master/cards/000096/files/2093_ruby_28087.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 421747 (412K) [application/zip]
Saving to: ‘2093_ruby_28087.zip’

2093_ruby_28087.zip 100%[===================>] 411.86K  --.-KB/s    in 0.03s   

2020-11-16 13:57:56 (11.9 MB/s) - ‘

ファイルを読み込みます

In [2]:
text_list = []
with open('dogura_magura.txt', encoding='shift_jis') as f:
  text_list = f.readlines()

In [3]:
text_list[0:10]

['ドグラ・マグラ\n',
 '夢野久作\n',
 '\n',
 '-------------------------------------------------------\n',
 '【テキスト中に現れる記号について】\n',
 '\n',
 '《》：ルビ\n',
 '（例）蜜蜂《みつばち》\n',
 '\n',
 '｜：ルビの付く文字列の始まりを特定する記号\n']

## データの前処理


不要な文字の削除

In [4]:
import re
def normalize_text(text):
  text = re.sub(r'》', '', text)
  text = re.sub(r'※', '', text)
  text = re.sub(r'《', '', text)
  text = re.sub(r'［', '', text)
  text = re.sub(r'＃', '', text)
  text = re.sub(r'-', '', text)
  text = re.sub(r'｜', '', text)
  text = re.sub(r'］', '', text)
  text = re.sub(r'［', '', text)
  text = re.sub(r'【','', text)
  text = re.sub(r'】','', text)
  text = text.strip()
  return text

In [5]:
new_text_list = []
for text in text_list:
  text = normalize_text(text)
  new_text_list.append(text)

text_list = new_text_list

In [6]:
text_list[0:10]

['ドグラ・マグラ',
 '夢野久作',
 '',
 '',
 'テキスト中に現れる記号について',
 '',
 '：ルビ',
 '（例）蜜蜂みつばち',
 '',
 '：ルビの付く文字列の始まりを特定する記号']

形態素解析ライブラリの「janome」をインストール

In [7]:
!pip install janome

     |████████████████████████████████| 19.7MB 1.3MB/s 


分かち書き

In [8]:
from janome.tokenizer import Tokenizer

In [9]:
def wakachigaki(text_list):
  t = Tokenizer()
  words = []
  for text in text_list:
    tokens = t.tokenize(text)
    for token in tokens:
      pos = token.part_of_speech.split(',')[0]
      words.append(token.surface)

  text = ' '.join(words)
  return text

In [10]:
word_list = [w for w in wakachigaki(text_list).split()]

In [11]:
word_list[0:10]

['ドグラ・マグラ', '夢野', '久作', 'テキスト', '中', 'に', '現れる', '記号', 'について', '：']

## マルコフ連鎖モデルの学習

### 簡単な例で確認

In [12]:
test_text = ['私はからあげが好きだ。君はからあげを食べる。私はおやつが好きだ。']
test_text = wakachigaki(test_text)
test_text = test_text.replace('から あげ', 'からあげ')
test_text = test_text.replace('お やつ', 'おやつ')

test_word_list = [w for w in test_text.split()]
test_word_list

['私',
 'は',
 'からあげ',
 'が',
 '好き',
 'だ',
 '。',
 '君',
 'は',
 'からあげ',
 'を',
 '食べる',
 '。',
 '私',
 'は',
 'おやつ',
 'が',
 '好き',
 'だ',
 '。']

一階のマルコフ連鎖モデル

In [13]:
def make_markov_model_1(word_list):
  markov = {}
  w1 = ''
  for word in word_list:
    if w1:
      if w1 not in markov:
        markov[w1] = []
      markov[w1].append(word)
    w1 = word
  return markov  

In [14]:
markov_model_test_1 = make_markov_model_1(test_word_list)

学習したモデルの確認

In [15]:
def check_model(model, check_numb=10):
  count = 0
  for key in model.keys():
    if count >= 0:
      print('key:', key)
      print('value:', model[key])
      print('------------------------')
    count += 1
    if count > check_numb:
      break

In [16]:
check_model(markov_model_test_1, check_numb=20)

key: 私
value: ['は', 'は']
------------------------
key: は
value: ['からあげ', 'からあげ', 'おやつ']
------------------------
key: からあげ
value: ['が', 'を']
------------------------
key: が
value: ['好き', '好き']
------------------------
key: 好き
value: ['だ', 'だ']
------------------------
key: だ
value: ['。', '。']
------------------------
key: 。
value: ['君', '私']
------------------------
key: 君
value: ['は']
------------------------
key: を
value: ['食べる']
------------------------
key: 食べる
value: ['。']
------------------------
key: おやつ
value: ['が']
------------------------


In [17]:
test_text = ['私はからあげが好きだ。君はからあげを食べる。私はおやつが好きだ。空が青い。']
test_text = wakachigaki(test_text)
test_text = test_text.replace('から あげ', 'からあげ')
test_text = test_text.replace('お やつ', 'おやつ')

test_word_list = [w for w in test_text.split()]
test_word_list

['私',
 'は',
 'からあげ',
 'が',
 '好き',
 'だ',
 '。',
 '君',
 'は',
 'からあげ',
 'を',
 '食べる',
 '。',
 '私',
 'は',
 'おやつ',
 'が',
 '好き',
 'だ',
 '。',
 '空',
 'が',
 '青い',
 '。']

In [18]:
markov_model_test_1 = make_markov_model_1(test_word_list)

In [19]:
check_model(markov_model_test_1, check_numb=20)

key: 私
value: ['は', 'は']
------------------------
key: は
value: ['からあげ', 'からあげ', 'おやつ']
------------------------
key: からあげ
value: ['が', 'を']
------------------------
key: が
value: ['好き', '好き', '青い']
------------------------
key: 好き
value: ['だ', 'だ']
------------------------
key: だ
value: ['。', '。']
------------------------
key: 。
value: ['君', '私', '空']
------------------------
key: 君
value: ['は']
------------------------
key: を
value: ['食べる']
------------------------
key: 食べる
value: ['。']
------------------------
key: おやつ
value: ['が']
------------------------
key: 空
value: ['が']
------------------------
key: 青い
value: ['。']
------------------------


In [20]:
def make_markov_model_2(text_list):
  markov = {}
  w1 = ''
  w2 = ''
  for word in text_list:
    if w1 and w2:
      if (w1, w2) not in markov:
        markov[(w1, w2)] = []
      markov[(w1, w2)].append(word)
    w1, w2 = w2, word
  return markov  

In [21]:
markov_model_test_2 = make_markov_model_2(test_word_list)

In [22]:
check_model(markov_model_test_2, check_numb=20)

key: ('私', 'は')
value: ['からあげ', 'おやつ']
------------------------
key: ('は', 'からあげ')
value: ['が', 'を']
------------------------
key: ('からあげ', 'が')
value: ['好き']
------------------------
key: ('が', '好き')
value: ['だ', 'だ']
------------------------
key: ('好き', 'だ')
value: ['。', '。']
------------------------
key: ('だ', '。')
value: ['君', '空']
------------------------
key: ('。', '君')
value: ['は']
------------------------
key: ('君', 'は')
value: ['からあげ']
------------------------
key: ('からあげ', 'を')
value: ['食べる']
------------------------
key: ('を', '食べる')
value: ['。']
------------------------
key: ('食べる', '。')
value: ['私']
------------------------
key: ('。', '私')
value: ['は']
------------------------
key: ('は', 'おやつ')
value: ['が']
------------------------
key: ('おやつ', 'が')
value: ['好き']
------------------------
key: ('。', '空')
value: ['が']
------------------------
key: ('空', 'が')
value: ['青い']
------------------------
key: ('が', '青い')
value: ['。']
------------------------


### ドグラ・マグラを使ってモデルを生成

In [23]:
markov_model_2 = make_markov_model_2(word_list)

In [24]:
check_model(markov_model_2, check_numb=20)

key: ('ドグラ・マグラ', '夢野')
value: ['久作']
------------------------
key: ('夢野', '久作')
value: ['テキスト', '全集']
------------------------
key: ('久作', 'テキスト')
value: ['中']
------------------------
key: ('テキスト', '中')
value: ['に']
------------------------
key: ('中', 'に')
value: ['現れる', 'ハッキリ', '、', '、', '走り込み', 'ヨボヨボ', '含ま', 'は', '納まっ', '凝固', 'ゴチャゴチャ', '落ち', '、', '追い込ん', '映っ', '突立', '、', '誘い込ん', '湧き', '落し', '並ん', 'は', '封じ込め', '、', '一つ', '、', 'は', '盛込ま', '、', '記述', '記述', '陳列', '含ま', '、', '潜伏', '引寄', 'タッタ', '含ま', 'は', 'は', '摘発', '、', '…', 'は', '落し', '胚胎', '尊', '。', 'は', 'も', '公', '並ぶ', 'は', '交', 'さまよう', '重なる', '。', '居る', 'トグロ', '立て', '立ち止まっ', '発見', '宿っ', '在る', '呼吸', '、', 'ホッ', '、', '、', '葬り', '立往生', '詰めかけ', 'は', '引っ', 'も', '平等', '往々', '、', '見知っ', 'は', '、', '潜在', '寝', '潜在', '包み込ん', '現われ', '含ま', '咲く', '辛', '刺', '落付', '蝨', 'サッと', '留め', '含ま', 'は', '並べ立て', '据え', 'は', '突込', 'は', 'は', '、', 'は', '居る', '居る', '立ち', 'ベタ', 'も', '、', '現われ', '、', 'は', '隠れ', 'は', '息', '盛ら', '、', '潜ん', '納まっ', '一', 'も', '潜ん', 'も', 'は'

## 文章の生成

### 2階のマルコフモデルで文章生成
作成したモデルを使って文章を生成する

In [25]:
import random

def generate_text_2(model, max_sentence):
  count_sentence = 0
  sentence = ''
  w1, w2  = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[(w1, w2)])
      sentence += tmp
      if(tmp=='。'):
        count_sentence += 1
        sentence += '\n'
      w1, w2 = w2, tmp
    except:
      w1, w2  = random.choice(list(model.keys()))

  return sentence

In [26]:
print(generate_text_2(markov_model_2, 10))

とを描きあらわした……そもそも世界歴史というものはタッタ一粒の中ですから、姪の浜の呉一郎のその方面の専門のスタイナハでもあるまいかと思うと、云ひも終らず一間余り走り退のくよと思案のあげくが。
世界各地の博士は南側の窓に近づいて来る。
殊に肝きもたまの小使室で見ましたが、まだシツケの掛かっております。
「ヘイヘイ、今日の御家老職、雲井喜三郎、如何に深遠微妙な彩色や線の働らきつゝ人馬の傍で突然に発狂したままジッと我慢して説明すると別人では御座いませんでした。
そうしてコンナに成長おおきくなった人の妾たちが、九大の狂人以上にこのような、又は八犬伝や水滸伝すいこでんに出て、何の役に立つのか』……」私は帽子を慌てて外套の老婆に変って来ました黒怪人物は御座いませぬ私が何が胎児を主人公とする精神科学応用の犯罪……という事実について今更めかしく疑いを抱いて調べてみると、吾々の大昔から初まった正木博士投身自殺を思い立った最初の宣言を高唱する。
栄養のいい頭をチョットばかり竪たてに振った。
その中うちから、絵巻物を焼いたと見えていたが二人の名刺を二本ともダラリと垂れましたかはっきり記憶おぼえてしまっているモノスゴイ暗示材料を、ここいらで、色々な事で、大きな椅子の横で畠を打っております私共姉妹きょうだいに生き写しなんていう事は無論私に接近し初めた……そのような気が付く。
すなはち従ひ来れる馬士まごをたよりに、あとから吹き上げると、途中で理由も、既に化石となった姿は残っていた私の周囲まわりに立ち帰って行ったもので、何かしら考え込んでいない事を暗示するものを読んでいるだろう。
夜が明けてみると、そのうつむいたのである。
この絵巻物がさながら生きて生き甲斐ないこの病気の治療場の入口を開いた。



### 2階から5階のマルコフ連鎖に変更

In [27]:
def make_markov_model_5(word_list):
  markov = {}
  w1 = ''
  w2 = ''
  w3 = ''
  w4 = ''
  w5 = ''
  for word in word_list:
    if w1 and w2 and w3 and w4 and w5:
      if (w1, w2, w3, w4, w5) not in markov:
        markov[(w1, w2, w3, w4, w5)] = []
      markov[(w1, w2, w3, w4, w5)].append(word)
    w1, w2, w3, w4, w5 = w2, w3, w4, w5, word
  return markov

In [28]:
markov_model_5 = make_markov_model_5(word_list)

In [29]:
check_model(markov_model_5, 20)

key: ('ドグラ・マグラ', '夢野', '久作', 'テキスト', '中')
value: ['に']
------------------------
key: ('夢野', '久作', 'テキスト', '中', 'に')
value: ['現れる']
------------------------
key: ('久作', 'テキスト', '中', 'に', '現れる')
value: ['記号']
------------------------
key: ('テキスト', '中', 'に', '現れる', '記号')
value: ['について']
------------------------
key: ('中', 'に', '現れる', '記号', 'について')
value: ['：']
------------------------
key: ('に', '現れる', '記号', 'について', '：')
value: ['ルビ']
------------------------
key: ('現れる', '記号', 'について', '：', 'ルビ')
value: ['（']
------------------------
key: ('記号', 'について', '：', 'ルビ', '（')
value: ['例']
------------------------
key: ('について', '：', 'ルビ', '（', '例')
value: ['）']
------------------------
key: ('：', 'ルビ', '（', '例', '）')
value: ['蜜蜂']
------------------------
key: ('ルビ', '（', '例', '）', '蜜蜂')
value: ['みつば']
------------------------
key: ('（', '例', '）', '蜜蜂', 'みつば')
value: ['ち']
------------------------
key: ('例', '）', '蜜蜂', 'みつば', 'ち')
value: ['：']
------------------------
key: ('）', '蜜蜂', 'みつば', 'ち',

In [30]:
import random

def generate_text_5(model, max_sentence):
  count_sentence = 0
  sentence = ''
  w1, w2, w3, w4, w5  = random.choice(list(model.keys()))
  while count_sentence < max_sentence:
    try:
      tmp = random.choice(model[(w1, w2, w3, w4, w5)])
      sentence += tmp
      if(tmp=='。'):
        count_sentence += 1
        sentence += '\n'
      w1, w2, w3, w4, w5 = w2, w3, w4, w5, tmp
    except:
      w1, w2, w3, w4, w5  = random.choice(list(model.keys()))

  return sentence

In [31]:
print(generate_text_5(markov_model_5, 10))

と、すぐに鍬を放り出して、近くの堤どてか草原くさばらの木蔭か軒下のきしたに行って弁当を使う。
それから約半刻はんとき……と申しますのは、取りも直さず心理遺伝のあらわれに相違ないので、結局するところ、一切の摩訶まか不思議な大活躍を演ずる事に相成ましたので、つまり只今から御紹介致します空前絶後的な怪事件の真相と申しますのは唯一つ、貴方が昔の御記憶を回復されて、その附属病院の工事と共に着々進捗しんちょくしつつある「狂人解放治療場」を見た人々は、誰でも狂人の散歩場ぐらいにしか思っていないようであるが、眼を近づけて見るとそれは見えるか見えぬ位の細かい彩糸いろいとや金銀の糸で、極く薄い絹地の目を拾いつつ、一寸大の唐獅子の群れを一匹毎ごとに色を変えて隙間すきまなく刺した物で、貴いものである事が判明したので又々大騒ぎとなり、福岡地方裁判所から熱海判事、松岡書記、福岡警察署より津川警部、長谷川警察医外一名、又、大学側からは若林学部長を初め川路かわじ、安楽あんらく、太田、西久保の諸教授、田中書記等が現場に駆け付けたが、検案の結果同博士は、同海岸水族館裏手の石垣の上に帽子と葉巻きの吸いさしを置き、診察服を着けたまま手足を狂人用鉄製の手枷足枷てかせあしかせを以て緊縛し、折柄の満潮に身を投じたものらしく、死後約三時間を経過しているので救急の法も施ほどこしようがなかった。
而しかして、この仮定を認むる時は、その性的衝動の危機の裡うちに眼覚めたる呉一郎が、かかる局面に立ちたる結果起したる、この種の頭脳特有の錯覚に非ざるなきやを疑い得る事。
＝同＝妹千世子が家出の原因は刺繍と絵画の修業を目的とせるものに外ほかならざる旨、繰返して弁明せるも、前項の疑点と照合する時は、尚、別の意味をも含まれいるものの如し。
すなわち千世子は、姉と共に同家に居りては、到底結婚の不可能なるべきを予感し、又は他国に於て、呉家の血統の男に生れたものならば、きっと正気を取り失いまして、親でも姉妹きょうだいでも、又は赤の他人でも、女でさえあれば殺すような事を致しますのだそうで、その由来ことわけを書いたものが、あのお寺にあるとか……ないとか云うておるようで御座いますが……」「僕が……精神病の治療……」「さようで……まことに奇妙な標題ですが……」私は思わず立竦たちすくんだ。
そういう正木博士の態度の中には、私を押え付けて動かさない或

# 参考リンク
- https://omedstu.jimdofree.com/2018/05/06/%E3%83%9E%E3%83%AB%E3%82%B3%E3%83%95%E9%80%A3%E9%8E%96%E3%81%AB%E3%82%88%E3%82%8B%E6%96%87%E6%9B%B8%E7%94%9F%E6%88%90/
- https://qiita.com/k-jimon/items/f02fae75e853a9c02127